# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

from IPython.core.display import HTML

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
# Loading the dataset
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

#nltk.download('brown')
data = nltk.corpus.brown
print("There are {} sentences in the data set".format(len(data.sents())))

There are 57340 sentences in the data set


In [4]:
# Split the dataset

training_set = data.tagged_sents()[:int(len(data.tagged_sents())*0.8)]
print("There are total number of {} sentences in the training set".format(len(training_set)))

test_set = data.tagged_sents()[int(len(data.tagged_sents())*0.8):]
print("There are total number of {} sentences in the test set".format(len(test_set)))

There are total number of 45872 sentences in the training set
There are total number of 11468 sentences in the test set


In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [4]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [5]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [6]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [7]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [48]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    
    output format:
                    out = {tag: {word: occurance}}
    """
    # Initialization default dict of list type
    dic = defaultdict(list)
    
    # Initializing counter
    cnt = Counter()
    
    # Counting the occurance Counter((tag, word): occurance)
    for i_seq_A_seq_B in zip(sequences_A, sequences_B):
        cnt[i_seq_A_seq_B] += 1
    
    # Convering the Counter to list - [((tag, word), occurance)]
    tag_word_count = list(cnt.items()) 
    
    # Making a the main structure - {tag: [{word: occurance}]}
    for (part_of_speech, word), occurance in tag_word_count:
        dic[part_of_speech].append({word: occurance})
    
    # Converting defaultdict to dictionary
    out = dict(dic.items())
    
    # Convering the list of dictionaries to dictionary of dictionaries - {tag: {word: occurance}}
    for i_POS in out:
        # Initializing an empty dictionary
        new_list = {}
        # Getting the dictionary of words and occurance of a specigfit POS
        words_and_occurances = dic[i_POS] 
        # Convering list of dictionaries to one dictionary
        for i_word_and_occurance in words_and_occurances:     
            new_list["".join(i_word_and_occurance.keys())] = list(i_word_and_occurance.values())[0]
            
        # Assiging to the new_list to 'out'
        out[i_POS] = new_list
        
    return out


# Calculate C(t_i, w_i)
tags = [i[1] for i in data.tagged_words()]
words = [i[0] for i in data.tagged_words()]
emission_counts = pair_counts(tags, words)

#assert len(emission_counts) == 472, \
#       "Uh oh. There should be 472 tags in your dictionary."
#assert max(emission_counts["NN"], key=emission_counts["NN"].get) == 'time', \
#       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

In [49]:
len(emission_counts)

472

In [50]:
emission_counts["NN"]["time"]

1547

In [51]:
max(emission_counts["NN"], key = emission_counts["NN"].get)

'time'

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [52]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

# Most Frequent Class Tagger (MFC Tagger)
class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name = tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table


# Most Frequent Count (MFC) table
mfc_table = dict()
for word, tags in emission_counts.items():
    mfc_table[word] = max(tags.keys(), key = lambda key: tags[key])

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

#assert len(mfc_table) == len(training_corpus.tagged_words()), ""
#assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
#assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

In [140]:
"""# Initialize an empty dictionary
mfc_table = dict()
# Iterate thorugh each tag and each words
for i_tag, words in emission_counts.items():
    # Iterate though (words, occurance)
    for index, (i_word, i_occurance) in enumerate(list(words.items())):
        try:
            mfc_table[i_word] = i_occurance
        except:
            if i_occurance > mfc_table[i_word]:
                mfc_table[i_word] = i_occurance"""

In [154]:
mfc_table

{'The': 'AT',
 'Fulton': 'NP-TL',
 'County': 'NN-TL',
 'Grand': 'JJ-TL',
 'Jury': 'NN-TL',
 'said': 'VBD',
 'Friday': 'NR',
 'an': 'AT',
 'investigation': 'NN',
 'of': 'IN',
 "Atlanta's": 'NP$',
 'recent': 'JJ',
 'primary': 'JJ',
 'election': 'NN',
 'produced': 'VBN',
 '``': '``',
 'no': 'AT',
 'evidence': 'NN',
 "''": "''",
 'that': 'CS',
 'any': 'DTI',
 'irregularities': 'NNS',
 'took': 'VBD',
 'place': 'NN',
 '.': '.',
 'jury': 'NN',
 'further': 'RBR',
 'in': 'IN',
 'term-end': 'NN',
 'presentments': 'NNS',
 'the': 'AT',
 'City': 'NN-TL',
 'Executive': 'JJ-TL',
 'Committee': 'NN-TL',
 ',': ',',
 'which': 'WDT',
 'had': 'HVD',
 'over-all': 'JJ',
 'charge': 'NN',
 'deserves': 'VBZ',
 'praise': 'NN',
 'and': 'CC',
 'thanks': 'NNS',
 'Atlanta': 'NP',
 'for': 'IN',
 'manner': 'NN',
 'was': 'BEDZ',
 'conducted': 'VBN',
 'September-October': 'NP',
 'term': 'NN',
 'been': 'BEN',
 'charged': 'VBN',
 'by': 'IN',
 'Superior': 'JJ-TL',
 'Court': 'NN-TL',
 'Judge': 'NN-TL',
 'Durwood': 'NP',
 'P

In [142]:
print("Most Frequent Count (MFC) Table: \n\n", mfc_table)

Most Frequent Count (MFC) Table: 

 {'The': 452, 'an': 1, 'no': 3, 'the': 26, 'a': 7, 'A': 1, 'every': 434, 'An': 6, 'Every': 1, 'No': 8, "th'": 1, "ever'": 1, 'Fulton': 7, 'Atlanta': 31, 'Grady': 1, 'Georgia': 1, 'Jackson': 1, 'Miller': 20, 'Colquitt': 1, 'Texas': 8, 'Dallas': 2, 'Beaumont': 2, 'Lamar': 1, 'Texan': 3, 'Wise': 2, 'Paris': 1, 'Oklahoma': 10, 'Rhode': 5, 'Massachusetts': 36, 'U.S.': 4, 'Denton': 1, 'York': 1, 'St.': 3, 'Louis': 60, 'Cook': 1, 'Republican': 32, 'Atlantic': 1, 'Viet': 1, 'Nam': 1, 'Lao': 14, 'Boston': 1, 'America': 120, 'Asia': 25, 'Providence': 1, 'Narragansett': 2, 'Sheraton-Biltmore': 3, 'Johnston': 17, 'Westfield': 4, 'Jersey': 2, 'Mack': 1, 'Kennedy': 130, 'Essex': 7, 'Orange': 1, 'Michigan': 16, 'Morris': 18, 'Brooklyn': 13, 'Feis': 1, 'Hunter': 5, 'U.': 24, 'S.': 1, 'Times-Picayune': 1, 'Orleans': 1, 'Miles': 3, 'Pennsylvania': 31, 'Columbia': 2, 'Lafayette': 7, 'Samoa': 1, 'Frankford': 1, 'Hughes': 25, 'Philadelphia': 1, 'S.P.C.A.': 1, 'Montgomery'

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [53]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in words else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [54]:
print(simplify_decoding([i[0] for i in test_set[0]], mfc_model))
print([i[1] for i in test_set[0]])

['<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', ',', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '<MISSING>', '.']
['AT', 'NN', 'VBD', 'IN', 'AT', 'JJ', 'NN', ',', 'CC', 'PPSS', 'MD', 'VB', 'PPO', 'IN', 'PPO', '.']


In [ ]:
for key in data.testing_set.keys[:1]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [55]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [56]:
total_words = []
total_tags = []

for i_sent in training_set:
    i_word = []
    i_tag = []
    for w, t in i_sent:
        i_word.append(w)
        i_tag.append(t)
    total_words.append(i_word)
    total_tags.append(i_tag)

In [57]:
how_many_sents = 10
mfc_training_acc = accuracy(total_words[:how_many_sents], total_tags[:how_many_sents], mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))
#assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."

training accuracy mfc_model: 13.38%


In [17]:
how_many_sents = len(total_words)
mfc_training_acc = accuracy(total_words[:how_many_sents], total_tags[:how_many_sents], mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))
#assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."

training accuracy mfc_model: 92.49%


In [127]:
total_words = []
total_tags = []

for i_sent in test_set:
    i_word = []
    i_tag = []
    for w, t in i_sent:
        i_word.append(w)
        i_tag.append(t)
    total_words.append(i_word)
    total_tags.append(i_tag)

In [19]:
mfc_testing_acc = accuracy(total_words, total_tags, mfc_model)

print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

#assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

testing accuracy mfc_model: 92.28%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmin}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [13]:
import itertools
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
        counts the number of occurrences of the value in the sequences list. The sequences
        collection should be a 2-dimensional array.

        For example, if the tag NOUN appears 275558 times over all the input sequences,
        then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO
    sequences = itertools.chain.from_iterable(sequences)
    
    # Initializing a counter
    cnt = Counter()
    
    # Counting all sequences - Counter({'NOUN': 220632, 'VERB': 146161})
    for i_seq in sequences:
        cnt[i_seq] += 1
    
    # Convering the Counter to list - [('ADV', 44877), ('NOUN', 220632)]
    word_count = list(cnt.items()) 
    
    # Getting the sum of all occurances (all sequences together) - Optional Maybe
    sum_corpus = 0
    for i_occurance in word_count:
        sum_corpus += i_occurance[1]
        
    # Converting word_count to a dictionary
    dictionary = dict()
    for seq, occurance in word_count:
        dictionary[seq] = occurance #/ sum_corpus
    
    return dictionary


# TODO: call unigram_counts with a list of tag sequences from the training set
total_words = []
total_tags = []

for i_sent in data.tagged_sents():
    i_word = []
    i_tag = []
    for w, t in i_sent:
        i_word.append(w)
        i_tag.append(t)
    total_words.append(i_word)
    total_tags.append(i_tag)

tag_unigrams = unigram_counts(total_tags)
#print(tag_unigrams)

#assert set(tag_unigrams.keys()) == data.training_set.tagset, \
#       "Uh oh. It looks like your tag counts doesn't include all the tags!"
#assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
#        "Hmmm...'X' is expected to be the least common class"
# assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
#        "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

In [33]:
tag_unigrams

{'AT': 97959,
 'NP-TL': 4019,
 'NN-TL': 13372,
 'JJ-TL': 4107,
 'VBD': 26167,
 'NR': 1566,
 'NN': 152470,
 'IN': 120557,
 'NP$': 2565,
 'JJ': 64028,
 '``': 8837,
 "''": 8789,
 'CS': 22143,
 'DTI': 2921,
 'NNS': 55110,
 '.': 60638,
 'RBR': 1182,
 ',': 58156,
 'WDT': 5539,
 'HVD': 4895,
 'VBZ': 7373,
 'CC': 37718,
 'IN-TL': 1477,
 'BEDZ': 9806,
 'VBN': 29186,
 'NP': 34476,
 'BEN': 2470,
 'TO': 14918,
 'VB': 33693,
 'RB': 36464,
 'DT': 8957,
 'PPS': 18253,
 'DOD': 1047,
 'AP': 9522,
 'BER': 4379,
 'HV': 3928,
 'DTS': 2435,
 'VBG': 17893,
 'PPO': 11181,
 'QL': 8735,
 'JJT': 1005,
 'ABX': 730,
 'NN-HL': 1471,
 'VBN-HL': 137,
 'WRB': 4509,
 'CD': 13510,
 'MD': 12431,
 'BE': 6360,
 'JJR': 1958,
 'VBG-TL': 133,
 'BEZ': 10066,
 'NN$-TL': 361,
 'HVZ': 2433,
 'ABN': 3010,
 'PN': 2573,
 'PPSS': 13802,
 'PP$': 16872,
 'DO': 1353,
 'NN$': 1480,
 'NNS-HL': 609,
 'WPS': 3924,
 '*': 4603,
 'EX': 2164,
 'VB-HL': 125,
 ':': 1558,
 '(': 2264,
 ')': 2273,
 'NNS-TL': 2226,
 'NPS': 1275,
 'JJS': 359,
 'RP': 

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [14]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    # Initalizing a counter
    cnt = Counter()
    
    # Iterating through sequence
    for i_seq in sequences:
        
        # The length should be bigger than 1
        if len(i_seq) > 1:
            
            # Interating though length of sequence
            for index in range(1, len(i_seq)):
                
                # Bigram
                bigram = (i_seq[index - 1], i_seq[index])
                
                # Adding bigram to counter
                if bigram not in cnt:
                    cnt[bigram] = 0
                else:
                    cnt[bigram] += 1
    
    # Convering the counter to a dictionary
    output = dict(cnt)
    
    return output


tag_bigrams = bigram_counts(total_tags)
#print(tag_bigrams)
#print(len(tag_bigrams))
#print(tag_bigrams[('NOUN', 'VERB')])

# assert len(tag_bigrams) == 144, \
#        "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
# assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
#        "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
# assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
#        "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

In [35]:
print("Bigram tags: \n\n", tag_bigrams)

Bigram tags: 

 {('AT', 'NP-TL'): 808, ('NP-TL', 'NN-TL'): 1505, ('NN-TL', 'JJ-TL'): 131, ('JJ-TL', 'NN-TL'): 1756, ('NN-TL', 'VBD'): 401, ('VBD', 'NR'): 112, ('NR', 'AT'): 34, ('AT', 'NN'): 48371, ('NN', 'IN'): 42251, ('IN', 'NP$'): 873, ('NP$', 'JJ'): 326, ('JJ', 'NN'): 28406, ('NN', 'NN'): 11645, ('NN', 'VBD'): 3305, ('VBD', '``'): 70, ('``', 'AT'): 492, ('NN', "''"): 2796, ("''", 'CS'): 51, ('CS', 'DTI'): 224, ('DTI', 'NNS'): 374, ('NNS', 'VBD'): 1420, ('VBD', 'NN'): 530, ('NN', '.'): 19856, ('NN', 'RBR'): 115, ('RBR', 'VBD'): 46, ('VBD', 'IN'): 4380, ('IN', 'NN'): 17224, ('NN', 'NNS'): 5984, ('NNS', 'CS'): 1080, ('CS', 'AT'): 5636, ('AT', 'NN-TL'): 2563, ('NN-TL', ','): 1616, (',', 'WDT'): 777, ('WDT', 'HVD'): 143, ('HVD', 'JJ'): 56, ('IN', 'AT'): 43270, ('NN', ','): 18278, (',', '``'): 1092, ('``', 'VBZ'): 30, ('VBZ', 'AT'): 1381, ('NN', 'CC'): 9060, ('CC', 'NNS'): 2023, ('NNS', 'IN'): 14503, ('NN-TL', 'IN-TL'): 1188, ('IN-TL', 'NP-TL'): 413, ('NP-TL', "''"): 99, ("''", 'IN'): 38

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [29]:
total_words = []
total_tags = []

for i_sent in data.tagged_sents():
    i_word = []
    i_tag = []
    for w, t in i_sent:
        i_word.append(w)
        i_tag.append(t)
    total_words.append(i_word)
    total_tags.append(i_tag)

In [30]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    # Interating through each sequece (sentences in here) and getting the very first senquence 
    start = [i_seq[0] for i_seq in sequences]
    
    # Initializing a counter
    cnt = Counter()
    
    # Counting the start sequence
    for i_start in start:
        cnt[i_start] += 1
    
    # Convering the Counter to list - [('NOUN', 8093), ('CONJ', 2817)]
    word_count = list(cnt.items()) 

    # Converting the list to a dictionary
    dictionary = dict()
    for i_seq, occurance in word_count:
        dictionary[i_seq] = occurance
    
    return dictionary

# TODO: Calculate the count of each tag starting a sequence
#tags = data.Y
tag_starts = starting_counts(total_tags)
#print(tag_starts['NOUN'])

# assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
# assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
# assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [31]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # Interating through each sequece (sentences in here) and getting the very last senquence 
    end = [i_seq[-1] for i_seq in sequences]
    
    # Initializing a counter
    cnt = Counter()
    
    # Counting the end sequence
    for i_end in end:
        cnt[i_end] += 1
    
    #print(cnt)
    # Convering the Counter to list - [(('VERB', 'thanked'), 6), (('PRON', 'him'), 2576)]
    word_count = list(cnt.items()) 

    # Converting the list to a dictionary
    dictionary = dict()
    for i_seq, occurance in word_count:
        dictionary[i_seq] = occurance
    
    return dictionary


# TODO: Calculate the count of each tag ending a sequence
#tags = data.Y
tag_ends = ending_counts(total_tags)
#print(tag_ends)

# assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
# assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
# assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [32]:
# Initializing a Hidden Markov Model
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

In [33]:
# Getting the tags and words
tags = total_tags
words = total_words

# Emission count
emission_counts = pair_counts([i[1] for i in data.tagged_words()], [i[0] for i in data.tagged_words()])

In [34]:
# Initializing an empty dictionary for our states
states = {}

# Iterating through the tags
for tag in list(np.unique([i[1] for i in data.tagged_words()])):
    tag = str(tag)
    # Initializing an empty dictionary for our emission probability
    emission_probabilities = dict()
    
    # Iterating through the emission_counts
    for word, occurance in emission_counts[tag].items():
        
        # Creating the emission probability
        emission_probabilities[word] = occurance / tag_unigrams[tag]
    
    # A discrete distribution, made up of characters and their probabilities assuming that these probabilities will sum to 1.0.
    tag_distribution = DiscreteDistribution(emission_probabilities) 
    
    # Make a state in HMM for holding the emission probability
    state = State(tag_distribution, name = tag)
    
    # Adding the state to our dictionary
    states[tag] = state
    
    # Adding the state to our HMM
    basic_model.add_state(state)

In [35]:
# Adding the start and end edges
# Iterating though the tags
for tag in list(np.unique([i[1] for i in data.tagged_words()])):
    try:
        tag = str(tag)
        # Getting the state of the tag
        state = states[tag]

        # Calculate the start probability
        start_probability = tag_starts[tag] / sum(tag_starts.values())
        # Add a transition from state a to state b
        basic_model.add_transition(basic_model.start, state, start_probability)

        # Calculate the end probability
        end_probability = tag_ends[tag] / sum(tag_ends.values())
        # Add a transition from state a to state b
        basic_model.add_transition(state, basic_model.end, end_probability)
    except:
        pass

In [36]:
# Adding the in between edges
# Iterating through tags
try:
    for tag_1 in [i[1] for i in data.tagged_words()]:

        # Getting the state of the tag
        state_1 = states[tag_1]

        # Initializing the sum_of_probabilities
        sum_of_probabilities = 0

        # Iterating through tags
        for tag_2 in [i[1] for i in data.tagged_words()]:

            # Getting the state of the second tag
            state_2 = states[tag_2]

            # Sticking two tags together
            bigram = (tag_1, tag_2)

            # Calculating the transition probability
            transition_probability = tag_bigrams[bigram] / tag_unigrams[tag_1]

            # Adding the calculated probability to our sum_of_probabilities
            sum_of_probabilities += transition_probability

            # Add two states and their transition probability to our HMM
            basic_model.add_transition(state_1, state_2, transition_probability)
except:
    pass

In [37]:
basic_model.bake()
print("Number of nodes or states: ", basic_model.node_count())
print("Number of edges: ", basic_model.edge_count())

Number of nodes or states:  57
Number of edges:  120


In [24]:
emission_counts

{'AT': {'The': 6725,
  'an': 3518,
  'no': 1575,
  'the': 62288,
  'a': 21824,
  'A': 1119,
  'every': 434,
  'An': 188,
  'Every': 56,
  'No': 230,
  "th'": 1,
  "ever'": 1},
 'NP-TL': {'Fulton': 10,
  'Atlanta': 4,
  'Grady': 4,
  'Georgia': 14,
  'Jackson': 2,
  'Miller': 2,
  'Colquitt': 1,
  'Texas': 17,
  'Dallas': 13,
  'Beaumont': 1,
  'Lamar': 2,
  'Texan': 1,
  'Wise': 1,
  'Paris': 4,
  'Oklahoma': 3,
  'Rhode': 100,
  'Massachusetts': 14,
  'U.S.': 56,
  'Denton': 1,
  'York': 301,
  'St.': 11,
  'Louis': 13,
  'Cook': 2,
  'Republican': 9,
  'Atlantic': 6,
  'Viet': 10,
  'Nam': 9,
  'Lao': 3,
  'Boston': 11,
  'America': 74,
  'Asia': 19,
  'Providence': 38,
  'Narragansett': 4,
  'Sheraton-Biltmore': 3,
  'Johnston': 4,
  'Westfield': 2,
  'Jersey': 20,
  'Mack': 1,
  'Kennedy': 10,
  'Essex': 5,
  'Orange': 5,
  'Michigan': 5,
  'Morris': 3,
  'Brooklyn': 16,
  'Feis': 1,
  'Hunter': 3,
  'U.': 36,
  'S.': 38,
  'Times-Picayune': 1,
  'Orleans': 39,
  'Miles': 1,
  'Pen

In [59]:
how_many_sents = 10
mfc_training_acc = accuracy(total_words[:how_many_sents], total_tags[:how_many_sents], basic_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))
#assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."

training accuracy mfc_model: 0.00%


In [122]:
hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

AttributeError: 'CategorizedTaggedCorpusReader' object has no attribute 'testing_set'

In [25]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_training_acc > 0.955, "Uh oh. Your HMM accuracy on the training set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 96.18%


### Example Decoding Sequences with the HMM Tagger

In [26]:
for key in data.testing_set.keys[:1]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')





## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [27]:
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook HMM Tagger.ipynb to html',
 '[NbConvertApp] Writing 6224207 bytes to HMM Tagger.html']

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [28]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]

[nltk_data] Downloading package brown to
[nltk_data]     /Users/soheilmohammadpour/nltk_data...
[nltk_data]   Package brown is already up-to-date!


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

## Prediction